In [ ]:
from config import *
from main import *
from plot_utils import *
import itertools


In [ ]:
n_epochs = 10

env = gym.make('gym_nas_pt:nas_pt-v0', max_index=8, ch='all', sub='all', classifier='LSTM', use_redef_reward=False,
                   n_epochs_train=n_epochs, for_predictor=False)

In [ ]:
input_dim = n_epochs * 5 + 2

In [ ]:

Agent = DiscreteRNNAgent(1,env.action_size,env.nsc_space,seed=1,
                         n_kernels_conv=[32,64,128,256],kernel_sizes_conv=[1, 2, 3, 5, 8],
                         kernel_sizes_pool=[2,3,5], reward_shaping=True, use_predictor=False,
                         load_path=None, pred_input_dim=None, epsilon_schedule='20ksked')

In [ ]:
scores, scores_by_net = interact(Agent,env,n_episodes=20000, return_avg=5)
env.save_evaluated_models()

In [ ]:
mem = Agent.memory.memory
mem


In [ ]:
# plot the scores
plot_scores(scores)
import pickle as pickle
with open('scores_lstm_32_nodropout', 'wb') as file:
    pickle.dump(scores, file)
plt.plot(Agent.epsilon_schedule[:20000])

In [ ]:
results = pd.DataFrame.from_dict(env.evaluated_models, orient='index', columns = ['Criteria', 'Stdev', 'Inference Time', 'Density', 'FLOPs'])






score_df = pd.DataFrame.from_dict(scores_by_net, columns=['Score'], orient='index')

results = results[['Criteria', 'Density', 'Inference Time', 'FLOPs']].dropna()
results = results.join(score_df)

results_sorted = results.fillna(0).sort_values(['Score'])
print(results_sorted)

In [ ]:
from IPython.display import Image, display

In [ ]:
for i in range(10):
    best_model = [obj for obj in results_sorted.tail(10).index[i]]
    print(best_model)
    best_model = env.get_spec(best_model)
    env.render(best_model, ftype='png')

    display(Image(filename='output.png'))

In [ ]:
criteria = [[None for i in range(10)] for _ in range(5)]
inftime = [[None for i in range(10)] for _ in range(5)]
density = [[None for i in range(10)] for _ in range(5)]
flops = [[None for i in range(10)] for _ in range(5)]
targets = [[None for i in range(10)] for _ in range(5)]
pred = [[None for i in range(10)] for _ in range(5)]

for j in range(5):
    for i in range(10):
        model = [obj for obj in results_sorted.tail(10).index[i]]
#         model = [obj for obj in results_sorted.tail(2).iloc[i]['Model Specification']]
        model = env.get_spec(model)
        env.render(model, ftype='png')
        display(Image(filename='output.png'))
        try:
            criteria[j][i], inftime[j][i], density[j][i], flops[j][i], targets[j][i], pred[j][i] = env.test(model, verbosity='silent')
        except RuntimeError:
            criteria[j][i], inftime[j][i], density[j][i], flops[j][i], targets[j][i], pred[j][i] = 0,0,0,0,0,0
        print(np.array([criteria, flops, inftime]).T)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from seaborn import heatmap
opp_class_names = ['Null','Open Door 1','Open Door 2','Close Door 1','Close Door 2','Open Fridge',
'Close Fridge','Open Dishwasher','Close Dishwasher','Open Drawer 1','Close Drawer 1','Open Drawer 2','Close Drawer 2',
'Open Drawer 3','Close Drawer 3','Clean Table','Drink from Cup','Toggle Switch']
for i in range(10):
    mat = confusion_matrix(targets[i],pred[i], normalize='true')
    df_cm = pd.DataFrame(mat, index=opp_class_names,columns=opp_class_names)
    plt.figure(10,figsize=(15,12))
    heatmap(df_cm,annot=True,fmt='.2f',cmap='Purples')
    plt.show()

    print(classification_report(targets[i],pred[i]))


In [ ]:
print(results_sorted.tail(10)['Redef_rewards'].values)

In [ ]:
def generate_best_arch(agent, env):
    best_model = [(0,0,0,0,0,0)]
    done = False
    state = env.reset()
    score = 0
    while not done:
        agent.epsilon = 0
        agent.episode_counter = 0
        action = agent.act(state)
        print(action)
    # If action is valid, step the env, otherwise discard and gen another layer
        if env.action_to_nsc(action).valid():
            best_model.append(env.action_to_nsc(action).tuple)
            next_state, reward, done, _ = env.step(action, eval=False)
            agent.step(state, action, reward, next_state, done)
            state = next_state
    return best_model
    

In [ ]:

best_model = generate_best_arch(Agent, env)
best_model = env.get_spec(best_model) 


In [ ]:

env.render(best_model, ftype='png')
from IPython.display import Image
Image(filename='output.png')

In [ ]:
criteria, inftime, density, flops, targets, pred = env.test(best_model)

In [ ]:
print(criteria)

In [ ]:
deepconv = ((0,0,0,0,0,0),(1,1,5,0,0,64),(2,1,5,1,0,64),(3,1,5,2,0,64),(4,1,5,3,0,64),(5,4,0,0,0,0))
deepconv = env.get_spec(deepconv)
criteria, inftime, density, flops, targets, pred = env.test(deepconv)
print(criteria, flops, inftime)

In [ ]:
maxff = ((0,0,0,0,0,0),(1,1,5,0,0,256),(2,1,5,1,0,256),(3,1,5,2,0,256),(4,1,5,3,0,256),(5,1,5,4,0,256),(6,1,5,5,0,256),(7,1,5,6,0,256),(8,1,5,7,0,256),(9,4,0,0,0,0))
maxff = env.get_spec(maxff)
criteria, inftime, density, flops, targets, pred = env.test(maxff)
print(criteria, flops, inftime)

In [ ]:
maxwide = ((0,0,0,0,0,0),(1,1,5,0,0,256),(2,1,5,0,0,256),(3,1,5,0,0,256),(4,1,5,0,0,256),(5,1,5,0,0,256),(6,1,5,0,0,256),(7,1,5,0,0,256),(8,1,5,0,0,256),(9,4,0,0,0,0))
maxwide = env.get_spec(maxwide)
criteria, inftime, density, flops, targets, pred = env.test(maxwide)
print(criteria, flops, inftime)

In [ ]:
results.to_pickle('search_results_basel_10.pkl')